In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#https://github.com/youonf/recommendation_system/blob/master/content_based_filtering/content_based_recommender_approach1.ipynb
#https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243

In [2]:
songs_df = pd.read_csv('Data/songs_w_genres_df.csv', index_col='id')
songs_df.head()

,name,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
id,,,,,,,,,,,,,,,,,,
0cS0A1fUEUd1EW3FcF8AEI,keep a song in your soul,Mamie Smith,1920,0,12,168333,0.991000,0.598,0.224,0.000522,0.3790,-12.628,0,F,0.0936,149.976,0.6340,"['harlem renaissance', 'traditional blues']"
0hbkKFIJm7Z05H8Zl9w30f,i put a spell on you,Screamin' Jay Hawkins,1920,0,7,150200,0.643000,0.852,0.517,0.026400,0.0809,-7.261,0,F,0.0534,86.889,0.9500,"['blues', 'traditional blues']"
11m7laMUgmOKqI3oYzuhne,golfing papa,Mamie Smith,1920,0,4,163827,0.993000,0.647,0.186,0.000018,0.5190,-12.098,1,C,0.1740,97.600,0.6890,"['harlem renaissance', 'traditional blues']"
19Lc5SfJJ5O1oaxY0fpwfh,true house music - xavier santos & carlos gomi...,Oscar Velazquez,1920,0,17,422087,0.000173,0.730,0.798,0.801000,0.1280,-7.311,1,D,0.0425,127.997,0.0422,['circuit']
2hJjbsLCytGsnAHfdsLejp,xuniverxe,Mixe,1920,1,2,165224,0.295000,0.704,0.707,0.000246,0.4020,-6.036,0,A#,0.0768,122.076,0.2990,['glitchcore']


In [3]:
songs_df = songs_df.astype({'acousticness': 'float32', 'danceability': 'float32', 'energy': 'float32', 
                            'instrumentalness': 'float32', 'liveness': 'float32', 'loudness': 'float32', 
                            'speechiness': 'float32', 'tempo': 'float32', 'valence': 'float32',})

In [4]:
songs_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159101 entries, 0cS0A1fUEUd1EW3FcF8AEI to 3N3Wi5Un7iT8amLezSRwub
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              159101 non-null  object 
 1   artists           159101 non-null  object 
 2   year              159101 non-null  int64  
 3   explicit          159101 non-null  int64  
 4   popularity        159101 non-null  int64  
 5   duration_ms       159101 non-null  int64  
 6   acousticness      159101 non-null  float32
 7   danceability      159101 non-null  float32
 8   energy            159101 non-null  float32
 9   instrumentalness  159101 non-null  float32
 10  liveness          159101 non-null  float32
 11  loudness          159101 non-null  float32
 12  mode              159101 non-null  int64  
 13  key               159101 non-null  object 
 14  speechiness       159101 non-null  float32
 15  tempo             159101 non-null  f

In [5]:
songs_df.index

Index(['0cS0A1fUEUd1EW3FcF8AEI', '0hbkKFIJm7Z05H8Zl9w30f',
       '11m7laMUgmOKqI3oYzuhne', '19Lc5SfJJ5O1oaxY0fpwfh',
       '2hJjbsLCytGsnAHfdsLejp', '3HnrHGLE9u2MjHtdobfWl9',
       '5DlCyqLyX2AOVDTjjkDZ8x', '02FzJbHtqElixxCmrpSCUa',
       '02i59gYdjlhBmbbWhf8YuK', '06NUxS2XL3efRh0bloxkHm',
       ...
       '3NUmUIyzNLBp8bCFMH8Mif', '0fJ1caLzidzTlIL3pPX1eU',
       '0VCOLfLPC9oOrc90WGp2r3', '660rulYF3eLCuW6rQpiMdL',
       '0qQPTWtCFnAI4SucF4ytVb', '0sZ6HFSulsrOl6VTBwp6jd',
       '2LeqqwzobL5ktfQhWA3bHh', '3Glmyv3hbGGTgeR3FZrxJA',
       '6QGVWUbmlePAiY5zJjfCmT', '3N3Wi5Un7iT8amLezSRwub'],
      dtype='object', name='id', length=159101)

In [6]:
songs_df.shape

(159101, 18)

In [7]:
songs_df.genres = songs_df.genres.str[1:-1]
songs_df.head()

,name,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
id,,,,,,,,,,,,,,,,,,
0cS0A1fUEUd1EW3FcF8AEI,keep a song in your soul,Mamie Smith,1920,0,12,168333,0.991000,0.598,0.224,0.000522,0.3790,-12.628,0,F,0.0936,149.975998,0.6340,"'harlem renaissance', 'traditional blues'"
0hbkKFIJm7Z05H8Zl9w30f,i put a spell on you,Screamin' Jay Hawkins,1920,0,7,150200,0.643000,0.852,0.517,0.026400,0.0809,-7.261,0,F,0.0534,86.889000,0.9500,"'blues', 'traditional blues'"
11m7laMUgmOKqI3oYzuhne,golfing papa,Mamie Smith,1920,0,4,163827,0.993000,0.647,0.186,0.000018,0.5190,-12.098,1,C,0.1740,97.599998,0.6890,"'harlem renaissance', 'traditional blues'"
19Lc5SfJJ5O1oaxY0fpwfh,true house music - xavier santos & carlos gomi...,Oscar Velazquez,1920,0,17,422087,0.000173,0.730,0.798,0.801000,0.1280,-7.311,1,D,0.0425,127.997002,0.0422,'circuit'
2hJjbsLCytGsnAHfdsLejp,xuniverxe,Mixe,1920,1,2,165224,0.295000,0.704,0.707,0.000246,0.4020,-6.036,0,A#,0.0768,122.075996,0.2990,'glitchcore'


In [8]:
songs_df.head()['genres'].str.get_dummies(sep = ",")

,'traditional blues','blues','circuit','glitchcore','harlem renaissance'
id,,,,,
0cS0A1fUEUd1EW3FcF8AEI,1,0,0,0,1
0hbkKFIJm7Z05H8Zl9w30f,1,1,0,0,0
11m7laMUgmOKqI3oYzuhne,1,0,0,0,1
19Lc5SfJJ5O1oaxY0fpwfh,0,0,1,0,0
2hJjbsLCytGsnAHfdsLejp,0,0,0,1,0


In [9]:
songs_1980s_up = songs_df[songs_df['year'] > 1980]
songs_1980s_up.year.unique()

array([2008, 2018, 2020, 1997, 1991, 2006, 2012, 2011, 2007, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1992, 1993, 1994,
       1995, 1996, 1998, 1999, 2000, 2019, 2013, 2001, 2016, 2010, 2002,
       2004, 2005, 2015, 2003, 2009, 2021, 2014, 2017], dtype=int64)

In [10]:
songs_1980s_up.reset_index(inplace=True)
songs_1980s_up.set_index(['id','name'], inplace=True)


In [11]:
numerical_songs = songs_1980s_up.select_dtypes(include=['float32'])
numerical_songs.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 72230 entries, ('6catF1lDhNTjjGa2GxRQNN', "you'll never walk alone - mono; 2002 remaster") to ('3N3Wi5Un7iT8amLezSRwub', 'a little more')
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      72230 non-null  float32
 1   danceability      72230 non-null  float32
 2   energy            72230 non-null  float32
 3   instrumentalness  72230 non-null  float32
 4   liveness          72230 non-null  float32
 5   loudness          72230 non-null  float32
 6   speechiness       72230 non-null  float32
 7   tempo             72230 non-null  float32
 8   valence           72230 non-null  float32
dtypes: float32(9)
memory usage: 8.1+ MB


In [12]:
'''songs_df1 = songs_df
d_list = []

for index, row in songs_df.iterrows():
    for value in str(row['genres']).split(','):
        d_list.append({'name': row['name'], 'value':value})
songs_df1 = songs_df1.append(d_list, ignore_index=True)
songs_df1 = songs_df1.groupby('name')['value'].value_counts()
songs_df1 = songs_df1.unstack(level=-1).fillna(0)'''

"songs_df1 = songs_df\nd_list = []\n\nfor index, row in songs_df.iterrows():\n    for value in str(row['genres']).split(','):\n        d_list.append({'name': row['name'], 'value':value})\nsongs_df1 = songs_df1.append(d_list, ignore_index=True)\nsongs_df1 = songs_df1.groupby('name')['value'].value_counts()\nsongs_df1 = songs_df1.unstack(level=-1).fillna(0)"

In [13]:
#songs_df1[songs_df1.index == 'i write sins not tragedies']

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


In [15]:
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(numerical_songs), columns=numerical_songs.columns, index=numerical_songs.index)
df_scaled.head()

,,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
id,name,,,,,,,,,
6catF1lDhNTjjGa2GxRQNN,you'll never walk alone - mono; 2002 remaster,0.395582,0.489879,0.265,0.000000,0.149,0.771679,0.033472,0.505995,0.285
4aSw1QJIMwYSoDEgzgdCJL,ferry cross the mersey - mono; 2002 remaster,0.256024,0.409919,0.365,0.000005,0.163,0.785488,0.030042,0.465770,0.588
0ZMMtH875IR2TfkyC4PolD,don't let the sun catch you crying (main) - mono,0.407631,0.482794,0.352,0.000000,0.122,0.723326,0.031185,0.475737,0.478
1hx7X9cMXHWJjknb9O6Ava,"the september of my years - live at the sands hotel and casino, las vegas/1966 / show 2",0.890562,0.322875,0.201,0.000000,0.904,0.666025,0.064761,0.521986,0.239
19oquvXf3bc65GSqtPYA5S,"it was a very good year - live at the sands hotel and casino, las vegas/1966 / show 2",0.941767,0.272267,0.129,0.000005,0.683,0.660154,0.059875,0.366838,0.160


In [16]:
indices = pd.Series(df_scaled.index.get_level_values(1))
indices

0            you'll never walk alone - mono; 2002 remaster
1             ferry cross the mersey - mono; 2002 remaster
2         don't let the sun catch you crying (main) - mono
3        the september of my years - live at the sands ...
4        it was a very good year - live at the sands ho...
                               ...                        
72225                                             i giorni
72226                                       nuvole bianche
72227                                          una mattina
72228                                                night
72229                                        a little more
Name: name, Length: 72230, dtype: object

# Cosine Similarity

In [17]:
#!free -m

In [18]:
#songs_1980s_genres = songs_1980s_up.dropna(subset=['genres'])
#songs_1980s_genres.genres.isna().value_counts()

In [19]:
#vect = CountVectorizer()
#X = vect.fit_transform(songs_1980s_genres['genres']) 
#X


In [20]:
#cosine_sim = cosine_similarity(X, X)
#cosine_sim

In [21]:
#Plug in all numerical values
songs_1980s_up_scaled = df_scaled[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 
                                        'loudness', 'speechiness', 'tempo', 'valence']]

cosine_sim = cosine_similarity(songs_1980s_up_scaled, songs_1980s_up_scaled)

cosine_sim


array([[1.        , 0.9579409 , 0.98383754, ..., 0.6780117 , 0.66664344,
        0.946824  ],
       [0.9579409 , 1.        , 0.9852171 , ..., 0.57576084, 0.55437684,
        0.863901  ],
       [0.98383754, 0.9852171 , 0.9999998 , ..., 0.65590614, 0.63507164,
        0.92551845],
       ...,
       [0.6780118 , 0.57576084, 0.65590614, ..., 1.0000001 , 0.9855557 ,
        0.7747918 ],
       [0.66664344, 0.5543769 , 0.6350716 , ..., 0.9855557 , 1.0000002 ,
        0.77905774],
       [0.94682395, 0.863901  , 0.9255184 , ..., 0.7747918 , 0.77905774,
        1.0000002 ]], dtype=float32)

In [24]:
def recommendations(name, cosine_sim=cosine_sim):
    #INPUT FOR ARTISTS RETURN LIST OF ARTISTS
    name = name.lower()
    idx = indices[indices == name].index[0]
    print(songs_1980s_up.iloc[idx][0])
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6]
    
    songs_index = [i[0] for i in similarity_scores]
    
    return songs_1980s_up.iloc[songs_index]
    

In [25]:
recommendations('One Time', cosine_sim)

Justin Bieber


,,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
id,name,,,,,,,,,,,,,,,,,
1xasoowGI2N5c4gykxKTlW,icarus,Bastille,2013,0,52,225040,0.07120,0.583,0.778,0.000001,0.0753,-6.578,0,E,0.0459,134.945007,0.673,"'metropopolis', 'modern rock', 'pop', 'pop roc..."
0oDabL89N3jO4qnGMANjAS,can't get enough of you baby,The Colourfield,1985,0,35,142000,0.00715,0.613,0.754,0.000000,0.0568,-9.961,1,C,0.0392,127.888000,0.688,"'dance rock', 'new romantic', 'new wave', 'new..."
6AdZANL6JwhLhj1WGKT31v,always late with your kisses,Dwight Yoakam,1987,0,33,130440,0.11700,0.669,0.813,0.000008,0.0692,-4.854,1,A,0.0398,136.248993,0.721,"'bakersfield sound', 'country', 'country rock'"
7nfGP5B3Pc508kEb6zL6r9,the dead heart - 2007 remastered,Midnight Oil,1987,0,55,311027,0.02820,0.637,0.759,0.000386,0.0621,-11.004,1,G,0.0295,136.223007,0.686,"'australian rock', 'dance rock', 'rock'"
2ZmYpqOpup0GhQ8CNXn4sW,just like me,Pat Benatar,1981,0,24,210907,0.08150,0.575,0.694,0.018900,0.0740,-12.351,1,E,0.0500,127.491997,0.659,"'album rock', 'classic rock', 'dance rock', 'h..."


# PCA with cosine similarity

In [25]:
from sklearn.decomposition import PCA


In [22]:
pca = PCA()
pca_fit = pca.fit_transform(df_scaled)
pca_fit

array([[ 0.3107919 , -0.17992392, -0.22775841, ...,  0.01252388,
        -0.02054426, -0.04564155],
       [ 0.07478655, -0.19177659, -0.06462984, ...,  0.13376063,
         0.04404446, -0.05773644],
       [ 0.21950923, -0.23024954, -0.08318975, ...,  0.07883651,
         0.00971255,  0.01338098],
       ...,
       [ 1.2402827 ,  0.29129472,  0.1994881 , ..., -0.06730625,
         0.03977066,  0.1383599 ],
       [ 1.2547724 ,  0.34383297,  0.09320963, ...,  0.10359193,
         0.0462822 ,  0.04212236],
       [ 0.64561135, -0.32019687, -0.2276784 , ...,  0.06750678,
        -0.03201765, -0.04989262]], dtype=float32)

In [23]:
cosine_sim = cosine_similarity(pca_fit, pca_fit)

cosine_sim


array([[1.0000001 , 0.67189544, 0.8946614 , ..., 0.47384453, 0.49406567,
        0.86012316],
       [0.67189544, 1.        , 0.8363348 , ..., 0.08221135, 0.07556417,
        0.470967  ],
       [0.8946614 , 0.8363348 , 0.9999999 , ..., 0.4146108 , 0.40392557,
        0.8236787 ],
       ...,
       [0.47384453, 0.08221135, 0.4146108 , ..., 1.        , 0.9793997 ,
        0.6516542 ],
       [0.49406567, 0.07556417, 0.40392557, ..., 0.9793997 , 0.9999999 ,
        0.68448013],
       [0.86012316, 0.470967  , 0.8236787 , ..., 0.6516542 , 0.68448013,
        1.0000001 ]], dtype=float32)

In [73]:
recommendations('One Time', cosine_sim)

Justin Bieber


,,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
id,name,,,,,,,,,,,,,,,,,
1xasoowGI2N5c4gykxKTlW,icarus,Bastille,2013,0,52,225040,0.07120,0.583,0.778,0.000001,0.0753,-6.578,0,E,0.0459,134.945007,0.673,"'metropopolis', 'modern rock', 'pop', 'pop roc..."
0oDabL89N3jO4qnGMANjAS,can't get enough of you baby,The Colourfield,1985,0,35,142000,0.00715,0.613,0.754,0.000000,0.0568,-9.961,1,C,0.0392,127.888000,0.688,"'dance rock', 'new romantic', 'new wave', 'new..."
6AdZANL6JwhLhj1WGKT31v,always late with your kisses,Dwight Yoakam,1987,0,33,130440,0.11700,0.669,0.813,0.000008,0.0692,-4.854,1,A,0.0398,136.248993,0.721,"'bakersfield sound', 'country', 'country rock'"
7nfGP5B3Pc508kEb6zL6r9,the dead heart - 2007 remastered,Midnight Oil,1987,0,55,311027,0.02820,0.637,0.759,0.000386,0.0621,-11.004,1,G,0.0295,136.223007,0.686,"'australian rock', 'dance rock', 'rock'"
2ZmYpqOpup0GhQ8CNXn4sW,just like me,Pat Benatar,1981,0,24,210907,0.08150,0.575,0.694,0.018900,0.0740,-12.351,1,E,0.0500,127.491997,0.659,"'album rock', 'classic rock', 'dance rock', 'h..."


# Euclidean Distance

In [17]:
from sklearn.metrics.pairwise import euclidean_distances


In [18]:
D = euclidean_distances(df_scaled, df_scaled)
D

array([[0.        , 0.36013266, 0.22136207, ..., 1.1583288 , 1.1515847 ,
        0.42511165],
       [0.36013266, 0.        , 0.21496494, ..., 1.3279524 , 1.334813  ,
        0.6718846 ],
       [0.22136207, 0.21496494, 0.        , ..., 1.1994582 , 1.2083106 ,
        0.4972328 ],
       ...,
       [1.1583288 , 1.3279524 , 1.1994582 , ..., 0.        , 0.26563823,
        0.9948124 ],
       [1.1515847 , 1.334813  , 1.2083106 , ..., 0.26563823, 0.        ,
        0.9652096 ],
       [0.42511165, 0.6718846 , 0.4972328 , ..., 0.9948124 , 0.9652096 ,
        0.        ]], dtype=float32)

In [22]:
def recommend_euclidean_distance(name):
    
    name = name.lower()

    idx = indices[indices == name].index[0]
    print(songs_1980s_up.iloc[idx][0])

    distance = list(enumerate(D[idx]))
    distance = sorted(distance, key=lambda x:x[1])
    distance = distance[1:6]
    
    songs_index = [i[0] for i in distance]
    
    return songs_1980s_up.iloc[songs_index]

In [23]:
recommend_euclidean_distance('One Time')

Justin Bieber


,,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
id,name,,,,,,,,,,,,,,,,,
3AzjcOeAmA57TIOr9zF1ZW,physical,Dua Lipa,2020,0,81,193829,0.01370,0.647,0.844,0.000658,0.1020,-3.756,1,C,0.0457,146.966995,0.746,"['dance pop', 'pop', 'pop dance', 'uk pop']"
645YBsxhFPSOKxGep3uHLv,love maze,BTS,2018,0,70,221529,0.04870,0.636,0.876,0.000000,0.0728,-4.652,1,A,0.0617,149.932007,0.759,"['k-pop', 'k-pop boy group']"
78FJd3BMjDQCA1Hh2QfKqS,voy a pasármelo bien,Hombres G,1989,0,56,235467,0.04360,0.663,0.840,0.000006,0.0491,-5.122,1,C,0.0434,148.018005,0.828,"['latin', 'latin alternative', 'latin pop', 'l..."
1T20tqHKrePjszGOdbFOEd,take a hint,"Victorious Cast', 'Victoria Justice', 'Elizabe...",2012,0,65,154000,0.00443,0.727,0.860,0.000000,0.0718,-2.712,1,D#,0.0641,133.994995,0.747,NaN
16Of7eeW44kt0a1M0nitHM,you make me feel... (feat. sabi),"Cobra Starship', 'Sabi",2011,0,68,215693,0.01910,0.668,0.857,0.000007,0.0385,-2.944,0,G,0.0535,131.959000,0.748,NaN


# PCA with Euclidean Distance

In [26]:
pca = PCA()
pca_fit = pca.fit_transform(df_scaled)
pca_fit

array([[ 0.3107919 , -0.17992392, -0.22775841, ...,  0.01252388,
        -0.02054426, -0.04564155],
       [ 0.07478655, -0.19177659, -0.06462984, ...,  0.13376063,
         0.04404446, -0.05773644],
       [ 0.21950923, -0.23024954, -0.08318975, ...,  0.07883651,
         0.00971255,  0.01338098],
       ...,
       [ 1.2402827 ,  0.29129472,  0.1994881 , ..., -0.06730625,
         0.03977066,  0.1383599 ],
       [ 1.2547724 ,  0.34383297,  0.09320963, ...,  0.10359193,
         0.0462822 ,  0.04212236],
       [ 0.64561135, -0.32019687, -0.2276784 , ...,  0.06750678,
        -0.03201765, -0.04989262]], dtype=float32)

In [27]:
D = euclidean_distances(pca_fit, pca_fit)
D

array([[0.        , 0.3601475 , 0.22136883, ..., 1.1583242 , 1.1515802 ,
        0.4251103 ],
       [0.3601475 , 0.        , 0.21497284, ..., 1.3279537 , 1.3348145 ,
        0.6718876 ],
       [0.22136883, 0.21497284, 0.        , ..., 1.1994545 , 1.2083058 ,
        0.4972303 ],
       ...,
       [1.1583242 , 1.3279537 , 1.1994545 , ..., 0.        , 0.2656383 ,
        0.9948123 ],
       [1.1515802 , 1.3348145 , 1.2083058 , ..., 0.2656383 , 0.        ,
        0.9652094 ],
       [0.4251103 , 0.6718876 , 0.4972303 , ..., 0.9948123 , 0.9652094 ,
        0.        ]], dtype=float32)

In [28]:
recommend_euclidean_distance('One Time')

Justin Bieber


,,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
id,name,,,,,,,,,,,,,,,,,
3AzjcOeAmA57TIOr9zF1ZW,physical,Dua Lipa,2020,0,81,193829,0.01370,0.647,0.844,0.000658,0.1020,-3.756,1,C,0.0457,146.966995,0.746,"['dance pop', 'pop', 'pop dance', 'uk pop']"
645YBsxhFPSOKxGep3uHLv,love maze,BTS,2018,0,70,221529,0.04870,0.636,0.876,0.000000,0.0728,-4.652,1,A,0.0617,149.932007,0.759,"['k-pop', 'k-pop boy group']"
78FJd3BMjDQCA1Hh2QfKqS,voy a pasármelo bien,Hombres G,1989,0,56,235467,0.04360,0.663,0.840,0.000006,0.0491,-5.122,1,C,0.0434,148.018005,0.828,"['latin', 'latin alternative', 'latin pop', 'l..."
1T20tqHKrePjszGOdbFOEd,take a hint,"Victorious Cast', 'Victoria Justice', 'Elizabe...",2012,0,65,154000,0.00443,0.727,0.860,0.000000,0.0718,-2.712,1,D#,0.0641,133.994995,0.747,NaN
16Of7eeW44kt0a1M0nitHM,you make me feel... (feat. sabi),"Cobra Starship', 'Sabi",2011,0,68,215693,0.01910,0.668,0.857,0.000007,0.0385,-2.944,0,G,0.0535,131.959000,0.748,NaN


# Pearson Correlation

In [29]:
from scipy.stats import pearsonr

In [30]:
df_scaled_array = df_scaled.values
len(df_scaled_array)

72230

In [31]:
def recommend_pearson(name):
    
    name = name.lower()
    idx = indices[indices == name].index[0]
    print(songs_1980s_up.iloc[idx][0])
    
    correlation = []
    
    for i in range(len(df_scaled_array)):
        correlation.append(pearsonr(df_scaled_array[idx], df_scaled_array[i]))
    
    correlation = list(enumerate(correlation))
    sorted_corr = sorted(correlation, reverse=True, key=lambda x: x[1])[1:6]
    print(sorted_corr)
    songs_index = [i[0] for i in sorted_corr]
    
    return songs_1980s_up.iloc[songs_index]

In [32]:
recommend_pearson('One time')

Justin Bieber


C:\Users\vtkurt13\anaconda3\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


[(51165, (0.999095350754021, 7.325475206759157e-11)), (37702, (0.998830394100245, 1.7996229584610915e-10)), (30997, (0.9985649857545476, 3.6806771602588465e-10)), (49495, (0.9984579287843234, 4.734250825435821e-10)), (17020, (0.9984097751265599, 5.271938196960027e-10))]


,,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
id,name,,,,,,,,,,,,,,,,,
2ZmYpqOpup0GhQ8CNXn4sW,just like me,Pat Benatar,1981,0,24,210907,0.08150,0.575,0.694,0.018900,0.0740,-12.351,1,E,0.0500,127.491997,0.659,"['album rock', 'classic rock', 'dance rock', '..."
0oDabL89N3jO4qnGMANjAS,can't get enough of you baby,The Colourfield,1985,0,35,142000,0.00715,0.613,0.754,0.000000,0.0568,-9.961,1,C,0.0392,127.888000,0.688,"['dance rock', 'new romantic', 'new wave', 'ne..."
6AdZANL6JwhLhj1WGKT31v,always late with your kisses,Dwight Yoakam,1987,0,33,130440,0.11700,0.669,0.813,0.000008,0.0692,-4.854,1,A,0.0398,136.248993,0.721,"['bakersfield sound', 'country', 'country rock']"
1xasoowGI2N5c4gykxKTlW,icarus,Bastille,2013,0,52,225040,0.07120,0.583,0.778,0.000001,0.0753,-6.578,0,E,0.0459,134.945007,0.673,"['metropopolis', 'modern rock', 'pop', 'pop ro..."
0ccE7u6mHbB1LaSc1ZIQWm,it's a shame about ray - remastered,The Lemonheads,1992,0,47,187040,0.00164,0.645,0.792,0.000126,0.0799,-8.671,1,A,0.0304,131.393005,0.705,"['alternative pop', 'alternative rock', 'bosto..."


In [302]:
song = songs_1980s_up.head(5)
song

,,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
id,name,,,,,,,,,,,,,,,,,
6catF1lDhNTjjGa2GxRQNN,you'll never walk alone - mono; 2002 remaster,Gerry & The Pacemakers,2008,0,55,160187,0.394,0.484,0.265,0.000000,0.149,-11.101000,1,C,0.0322,113.564003,0.285,"'adult standards', 'brill building pop', 'brit..."
4aSw1QJIMwYSoDEgzgdCJL,ferry cross the mersey - mono; 2002 remaster,Gerry & The Pacemakers,2008,0,39,141987,0.255,0.405,0.365,0.000005,0.163,-10.226000,0,F#,0.0289,104.536003,0.588,"'adult standards', 'brill building pop', 'brit..."
0ZMMtH875IR2TfkyC4PolD,don't let the sun catch you crying (main) - mono,Gerry & The Pacemakers,2008,0,34,157093,0.406,0.477,0.352,0.000000,0.122,-14.165000,1,C#,0.0300,106.773003,0.478,"'adult standards', 'brill building pop', 'brit..."
1hx7X9cMXHWJjknb9O6Ava,"the september of my years - live at the sands hotel and casino, las vegas/1966 / show 2",Frank Sinatra,2018,0,27,187333,0.887,0.319,0.201,0.000000,0.904,-17.796000,1,G,0.0623,117.153000,0.239,"'adult standards', 'easy listening', 'lounge'"
19oquvXf3bc65GSqtPYA5S,"it was a very good year - live at the sands hotel and casino, las vegas/1966 / show 2",Frank Sinatra,2018,0,26,236800,0.938,0.269,0.129,0.000005,0.683,-18.167999,0,G,0.0576,82.332001,0.160,"'adult standards', 'easy listening', 'lounge'"


In [303]:
songs_1980s_up.index[1][1]

'ferry cross the mersey - mono; 2002 remaster'

In [377]:
def create_top_5(song_df):
    #extract the songs with name id and artists with list of top 5 most similar songs
    df_song = pd.DataFrame(song_df['artists'])
    df_song.reset_index(inplace=True)
    df_song.set_index('id', inplace=True)
    top_5_dict = {}
    for i in range(len(song_df)):
        top_5 = {}
        recs = recommendations(song_df.index[i][1], cosine_sim)
        for j in range(len(recs)):
            top_5[recs.index[j][0]] = recs.index[j][1], recs.iloc[j][0]
        top_5_dict[song_df.index[i][0]] = top_5
    top_5_series = pd.Series(top_5_dict)
    top_5_series.rename('top_5', inplace=True)
    top_5_songs_df = df_song.merge(top_5_series, how='inner', on=None, left_on=None, right_on=None, left_index=True, right_index=True)
    return top_5_songs-df

In [370]:
#top_5_df = create_top_5(songs_1980s_up)
#top_5_df

,name,artists,top_5
6catF1lDhNTjjGa2GxRQNN,you'll never walk alone - mono; 2002 remaster,Gerry & The Pacemakers,{'3GBncAOigK1ICW0X5eBn1X': ('you're not drinki...
4aSw1QJIMwYSoDEgzgdCJL,ferry cross the mersey - mono; 2002 remaster,Gerry & The Pacemakers,"{'2aCqeD8sJNjZ28qdhKLRdl': ('jerusalem', 'Dan ..."
0ZMMtH875IR2TfkyC4PolD,don't let the sun catch you crying (main) - mono,Gerry & The Pacemakers,{'3LJOZalbQSkfWLb0y2kypN': ('look at miss ohio...
1hx7X9cMXHWJjknb9O6Ava,the september of my years - live at the sands ...,Frank Sinatra,{'6bgnQlDQKgni9fH7mEQ1sC': ('the september of ...
19oquvXf3bc65GSqtPYA5S,it was a very good year - live at the sands ho...,Frank Sinatra,{'5PMgXE4qIjKNNJdL125Zj4': ('not a day goes by...
...,...,...,...
0sZ6HFSulsrOl6VTBwp6jd,i giorni,"Ludovico Einaudi', 'Johannes Bornlöf",{'2JPjsCvn379yvhrQ3Mdlfv': ('it's the most won...
2LeqqwzobL5ktfQhWA3bHh,nuvole bianche,"Ludovico Einaudi', 'Johannes Bornlöf","{'389qtBoJVF0riHChgXgrkt': ('moaning myrtle', ..."
3Glmyv3hbGGTgeR3FZrxJA,una mattina,"Ludovico Einaudi', 'Johannes Bornlöf",{'0jOnZhF75V68VsBObWx2XO': ('the carnival of t...
6QGVWUbmlePAiY5zJjfCmT,night,"Ludovico Einaudi', 'Johannes Bornlöf",{'2xTDIl5Ph5p5VNqqzdRL7j': ('god! save as... (...


In [371]:
top_5_df[top_5_df['name'] == 'one time']

,name,artists,top_5
6eDApnV9Jdb1nYahOlbbUh,one time,Justin Bieber,"{'1xasoowGI2N5c4gykxKTlW': ('icarus', 'Bastill..."
1kKvq34tmMVU0rWbxysJM6,one time,"Myka 9', 'Factor Chandelier","{'1xasoowGI2N5c4gykxKTlW': ('icarus', 'Bastill..."


In [376]:
top_5_df.to_csv(r'C:\Users\vtkurt13\Springboard\CapstoneThreeKurstedt\Data\top_5_df.csv', index=True, header=True)